# importing the required libraries 

In [1]:
import re
import pandas as pd
import numpy as np
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\PREM
[nltk_data]     SHROFF\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# definition of functions

In [2]:
# Function to check if a line starts with date and time
def date_time(s):
    pattern = r'^([0-9]{1,2}/[0-9]{1,2}/[0-9]{2,4}), ([0-9]{1,2}:[0-9]{2})\u202F?(AM|PM|am|pm)? - '
    result = re.match(pattern, s)
    return bool(result)
    
# Function to find if the line contains a contact name and message
def find_contact(s):
    s=s.split(":")
    if len(s)==2:
        return True
    else:
        return False
        
def get_message(line):
    split_line = line.split(' - ')
    datetime_part = split_line[0]
    date, time = datetime_part.split(', ')
    
    author = None 
    message = ""  

    if len(split_line) > 1:
        message_part = " ".join(split_line[1:])
        if find_contact(message_part):
            split_message = message_part.split(": ")
            author = split_message[0]
            message = " ".join(split_message[1:])
        else:
            message = message_part  

    return date, time, author, message

In [3]:
# Processing the chat file
data = []
conversation ='chat.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()  # Skip the first line if it is a header

    message_buffer = []
    date, time, author = None, None, None
    
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        
        if date_time(line):
            if len(message_buffer)>0:            
                data.append([date, time, author, ' '.join(message_buffer)])
            message_buffer.clear()
            
            date, time, author, message = get_message(line)
            message_buffer.append(message)
        else:
            message_buffer.append(line)

# analyses the sentiments

In [4]:
# Create DataFrame
df = pd.DataFrame(data, columns=["Date", "Time", "author", "Message"])
df['Date'] = pd.to_datetime(df['Date'])
df = df.dropna()

C:\Users\PREM SHROFF\AppData\Local\Temp\ipykernel_2792\354178735.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'])


In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
df["positive"] = [sentiments.polarity_scores(i)["pos"] for i in df["Message"]]
df["negative"] = [sentiments.polarity_scores(i)["neg"] for i in df["Message"]]
df["neutral"] = [sentiments.polarity_scores(i)["neu"] for i in df["Message"]]

df.head(50) 

,Date,Time,author,Message,positive,negative,neutral
1,2024-02-27,9:37 am,+91 81696 18367,Hii,0.000,0.0,1.000
2,2024-02-27,9:37 am,+91 81696 18367,null,0.000,0.0,1.000
3,2024-02-27,9:38 am,+91 81696 18367,Bol kya huaa,0.000,0.0,1.000
4,2024-02-27,9:38 am,~🅿️®️€ Ⓜ️__💲🅰️®️®️🅾️F~,College gya tha kya,0.000,0.0,1.000
5,2024-02-27,9:38 am,+91 81696 18367,Nhai,0.000,0.0,1.000
6,2024-02-27,9:38 am,+91 81696 18367,Kya huaa,0.000,0.0,1.000
7,2024-02-27,9:39 am,~🅿️®️€ Ⓜ️__💲🅰️®️®️🅾️F~,Reval ka photo copy aaya kya,0.000,0.0,1.000
8,2024-02-27,9:39 am,+91 81696 18367,Nhai,0.000,0.0,1.000
9,2024-02-27,9:39 am,~🅿️®️€ Ⓜ️__💲🅰️®️®️🅾️F~,Jane wala hai kya tu,0.000,0.0,1.000
10,2024-02-27,9:39 am,+91 81696 18367,Nhai,0.000,0.0,1.000


In [6]:
x=df["positive"].sum()
y=df["negative"].sum()
z=df["neutral"].sum()

def score(a,b,c):
    if (a>b) and (a>c):
        print ("Positive")
    if (b>a) and(b>c):
        print("Negative")
    if (c>a) and (c>b):
        print("Neutral")
        
score(x,y,z)
        

Neutral
